### **Importing the required modules**

In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

### Locating the data file path

In [2]:
filepath = "../1. Data Scraping/output/data.json"

### Comverting the JSON data to Pandas DataFrame

In [ ]:
# Load JSON data
with open(filepath, "r") as file:
    data = json.load(file)

# Extracting data
rows = []
for date, details in data.items():
    x = details["aqi"]
    if( x != None ):
        if( x["value"] != -1):
            row = {"Date": date}  # Daily date
            for metric in details["metrics"]:

                row[metric["name"]] = metric["avg"]  # Daily avg pollutant values

            row["AQI"] = x["value"]
    rows.append(row)


# Convert to DataFrame
df = pd.DataFrame(rows)

# Display the first few rows
df.head()

,Date,PM2.5,PM10,NO2,NH3,SO2,CO,OZONE,AQI
0,2019-05-01T00:00:00Z,81,140,13,2,28,22,23,140
1,2019-05-02T00:00:00Z,129,214,14,2,17,25,25,214
2,2019-05-03T00:00:00Z,89,144,11,2,17,13,43,144
3,2019-05-04T00:00:00Z,-,-,-,-,-,20,23,23
4,2019-05-05T00:00:00Z,78,116,6,2,13,27,25,116


### Replacing the missing values with NaN and Saving the CSV file

In [ ]:
# Load the CSV file
df = pd.read_csv('AQI_data.csv')

# Replace '-' with NaN for proper numerical handling
df.replace('-', np.nan, inplace=True)

# Replace ' ' with NaN for proper numerical handling
df.replace(' ', np.nan, inplace=True)

# Save as CSV
df.to_csv("AQI_data.csv", index=False)

df.head()

,Date,PM2.5,PM10,NO2,NH3,SO2,CO,OZONE,AQI
0,2019-05-01T00:00:00Z,81.0,140.0,13.0,2.0,28.0,22.0,23.0,140
1,2019-05-02T00:00:00Z,129.0,214.0,14.0,2.0,17.0,25.0,25.0,214
2,2019-05-03T00:00:00Z,89.0,144.0,11.0,2.0,17.0,13.0,43.0,144
3,2019-05-04T00:00:00Z,NaN,NaN,NaN,NaN,NaN,20.0,23.0,23
4,2019-05-05T00:00:00Z,78.0,116.0,6.0,2.0,13.0,27.0,25.0,116


### Imputing the missing values

In [5]:
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Selecting only numeric columns (excluding 'Date')
numeric_cols = df.select_dtypes(include=['number']).columns

# Step 1: Apply Linear Interpolation for short gaps
df[numeric_cols] = df[numeric_cols].interpolate(method='linear')

# Step 2: Apply 7-day rolling mean for medium gaps
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].rolling(7, min_periods=1).mean())

# Step 3: Use KNN Imputer for long gaps
imputer = KNNImputer(n_neighbors=5)
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# Check if any missing values remain
df.isnull().sum()

Date     0
PM2.5    0
PM10     0
NO2      0
NH3      0
SO2      0
CO       0
OZONE    0
AQI      0
dtype: int64

In [ ]:
# Round all numerical pollutant columns to 2 decimal places
df[numeric_cols] = df[numeric_cols].round(2)

,Date,PM2.5,PM10,NO2,NH3,SO2,CO,OZONE,AQI
0,2019-05-01 00:00:00+00:00,81.0,140.0,13.0,2.0,28.0,22.0,23.0,140.0
1,2019-05-02 00:00:00+00:00,129.0,214.0,14.0,2.0,17.0,25.0,25.0,214.0
2,2019-05-03 00:00:00+00:00,89.0,144.0,11.0,2.0,17.0,13.0,43.0,144.0
3,2019-05-04 00:00:00+00:00,83.5,130.0,8.5,2.0,15.0,20.0,23.0,23.0
4,2019-05-05 00:00:00+00:00,78.0,116.0,6.0,2.0,13.0,27.0,25.0,116.0


In [ ]:
# Save as CSV
df.to_csv("AQI_data_cleaned_FLOAT.csv", index=False)

In [ ]:
# Convert all pollutant and AQI values to integers
df[numeric_cols] = df[numeric_cols].astype(int)

# Verify data types after conversion
df.dtypes

Date     object
PM2.5     int64
PM10      int64
NO2       int64
NH3       int64
SO2       int64
CO        int64
OZONE     int64
AQI       int64
dtype: object

### Final Dataset

In [12]:
# Save as CSV
df.to_csv("AQI_data_cleaned_INT.csv", index=False)

### Recalculating the AQI dataset

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('updated_aqi_dataset_cleaned.csv')  # Replace with your actual filename

# Update the 'AQI' column to be the max of pollutant values for each row
pollutant_cols = ['PM2.5', 'PM10', 'NO2', 'NH3', 'SO2', 'CO', 'OZONE']
df['AQI'] = df[pollutant_cols].max(axis=1)

# Optional: Save to new CSV if needed
df.to_csv('updated_aqi_dataset_cleaned.csv', index=False)

df = df[['Date', 'AQI']]
df.head()

,Date,AQI
0,2019-05-01 00:00:00+00:00,140
1,2019-05-02 00:00:00+00:00,214
2,2019-05-03 00:00:00+00:00,144
3,2019-05-04 00:00:00+00:00,130
4,2019-05-05 00:00:00+00:00,116


### Saving the Univariate Version of the dataset

In [4]:
# Load the dataset
df = pd.read_csv('updated_aqi_dataset_cleaned.csv')  # Replace with your actual filename
df = df[['Date', 'AQI']]
df.to_csv('aqi_dataset.csv', index=False)